# Exploración de las alarmas de ascensores de 2016

In [ ]:
# Load libraries
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
key = 'ASC_2016_HIST'

store = pd.HDFStore('../data/SRA.hf5', mode='a')
if key in store:
    print('... reading from hdf')
    alarmas = pd.read_hdf(store, key)
else:
    print('... reading from csv')
    alarmas = pd.read_csv('../data/SRA/Parciales originales/SRA_' + key + '.csv', sep=';', parse_dates=['Fecha Inicio', 'Fecha Solución'], encoding='windows-1252', low_memory=False)
    alarmas.to_hdf(store, key)
    
store.close()

In [ ]:
alarmas.head()

In [ ]:
alarmas.shape

In [ ]:
alarmas.dtypes

In [ ]:
# Let's see how many alerts there are by metro line
alarmas.groupby('Línea').size()

In [ ]:
alarmas['Línea'].value_counts().plot(kind='bar', figsize=(15, 10))
plt.show()

In [ ]:
alarmas['Línea'].unique()

In [ ]:
alarmas['Línea_number'] = alarmas['Línea'].map({'L1':1, 'L2':2, 'L3':3, 'L4':4, 'L5':5, 'L6':6, 'L7':7, 'L8':8, 'L9':9, 'L10':10, 
                                         'L11':11, 'L12':12, 'RAMAL OPERA-P.PIO':13, 'ML1':14, 'ML2':15, 'ML3':16})

alarmas.head()

In [ ]:
# Nueva variable: Tiempo de resolución
alarmas['Tiempo resolucion'] = alarmas['Fecha Solución'] - alarmas['Fecha Inicio']
alarmas['Tiempo resolucion'] = alarmas['Tiempo resolucion'].dt.total_seconds()
alarmas.head()

In [ ]:
alarmas['Tiempo resolucion'].describe()

In [ ]:
alarmas['Tiempo resolucion'].plot(kind='density')
plt.show()

In [ ]:
alarmas['Tiempo resolucion'].hist()
plt.show()

In [ ]:
# Observamos que las fechas de solución no están bien informadas (1669/339867)
# De momento nos quitamos las que están mal informadas
alarmas = alarmas[alarmas['Tiempo resolucion'] >= 0]
alarmas.head()

In [ ]:
alarmas['Tiempo resolucion'].plot(kind='box')
plt.show()

In [ ]:
alarmas.shape

In [ ]:
# Nueva variable: Día de la semana en la que se informa la alarma
alarmas['Dia semana informada'] = alarmas['Fecha Inicio'].dt.dayofweek

# Nueva variable: día de la semana en la que se da por resuelta la incidencia
alarmas['Dia semana resolucion'] = alarmas['Fecha Solución'].dt.dayofweek

alarmas.head()

In [ ]:
# Veamos si a simple vista el día de la semana influye en el número de alarmas
alarmas.groupby('Dia semana informada').size()

In [ ]:
# Veamos si a simple vista el día de la semana de inicio de la alarma influye en el tiempo de resolución
# Scatter day of week - resolution time
plt.scatter(alarmas['Dia semana informada'], alarmas['Tiempo resolucion'])
plt.show()

In [ ]:
# Veamos si la hora a la que se informa una incidencia y el tiempo de resolución están relacionados
# Primero definimos una nueva variable: Hora a la que se informa la alarma (en segundos)
alarmas['Hora informada(sec)'] = alarmas['Fecha Inicio'].dt.hour * 3600 + alarmas['Fecha Inicio'].dt.minute * 60 + alarmas['Fecha Inicio'].dt.second

alarmas.head()

In [ ]:
plt.scatter(alarmas['Hora informada(sec)'], alarmas['Tiempo resolucion'])
plt.show()

In [ ]:
# Veamoslo por línea
plt.scatter(alarmas['Hora informada(sec)'], alarmas['Tiempo resolucion'], c=alarmas['Dia semana informada'], alpha=0.5)
plt.show()

In [ ]:
# Veamos si la línea en la que se da la alarma influye en el tiempo de resolución
# Scatter line - resolution time
plt.scatter(alarmas['Línea_number'], alarmas['Tiempo resolucion'])
plt.show()

In [ ]:
# Veamos si dependiendo de la hora hay más o menos inciencias
alarmas['Hora informada(sec)'].hist()
plt.show()

In [ ]:
# Comprobamos que a simple vista no hay diferencia de unos días a otros
alarmas['Hora informada(sec)'].hist(by=alarmas['Dia semana informada'], layout=(1,7), figsize=(16,3))
plt.show()

In [ ]:
# Parece que las correctivas se informan por la noche a partir de las 00:00 principalmente. Las predictivas se informan
# a lo largo del día, principalmente en las horas en las que el metro está abierto
alarmas['Hora informada(sec)'].hist(by=alarmas['Predictiva/Correctiva'])
plt.show()

In [ ]:
# Queremos ver si está relacionada la hora de la incidencia con el código de la misma
plt.scatter(alarmas['Código Alarma'], alarmas['Hora informada(sec)'])
plt.show()

In [ ]:
# Veamos si el tiempo de resolución lo está
plt.scatter(alarmas['Código Alarma'], alarmas['Tiempo resolucion'])
plt.show()

In [ ]:
alarmas.groupby('Código Alarma').size()